<h3>Read in Kaggle df</h3>

In [ ]:
import pandas as pd
path_kaggle = r'C:\Users\Desktop\MSBA Coursework\Tools for Analytics\datafile'\amazon_com_extras.csv'

with open(path_kaggle):
    #Added fields since line 808 in csv has more than 6 cols and needs to be told to ignore extra data.
    
    fields=['ASIN','GROUP','FORMAT','TITLE','AUTHOR','PUBLISHER']
    df_kaggle = pd.read_csv(path_kaggle, encoding='latin1', dtype=str,usecols=fields)

<h3> Read in amazon_com_extras. </h3>

In [ ]:
import pandas as pd
import random 
import json
from bs4 import BeautifulSoup as bs
import numpy as np
import requests
import time

my_csvfile=r'C:\Users\Desktop\MSBA Coursework\Tools for Analytics\datafile\AZscrape.csv'
start_entry=48001
end_entry=63000 

<h3> Use ISBN 10 to scrape book descriptions</h3>
User Agent Code - Refer https://www.scrapehero.com/how-to-fake-and-rotate-user-agents-using-python-3/


In [ ]:
import requests
import numpy as np

def amazon_description(book_metadata):
    
    import random
    import time
    
    #New columns that will be added to book_metadata
    book_descriptions=list()
    book_price=list()
    book_rating=list()
    
    user_agent_list = [
           #Chrome
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
            #Firefox
            'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
            'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
            'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
            'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
            'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
            'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
        ]
    
    
    book_metadata['Description']=''
    book_metadata['Price']=''
    book_metadata['Rating']=''
    for i in range(len(book_metadata)):
        
        #User Agents to prevent bot detection
        user_agent=random.choice(user_agent_list)
        book_isbn=book_metadata['ASIN'].iloc[i]
        
        print(book_isbn)
        base_url = "https://www.amazon.com/s/ref=nb_sb_noss?url=search-alias%3Dstripbooks-intl-ship&field-keywords="
        url=base_url+book_isbn
        print(url)
        headers={'User-Agent':user_agent}
        book_response=requests.get(url,headers=headers)
        if book_response.status_code != 200:
            print("Getting Book Details Failed")
            return None
        else:
            result_page=bs(book_response.content,'lxml')
            all_book_a_tags=result_page.find("a",class_="a-link-normal s-access-detail-page s-color-twister-title-link a-text-normal")
            try:
                book_url=all_book_a_tags.get("href")
                description,price,rating=get_book_description(book_url,user_agent_list)
            except: 
                continue
            print(description,price,rating)
#             book_descriptions.append(description)
#             book_price.append(price)
#             book_rating.append(rating)
        book_metadata['Description'].iloc[i]=description
        book_metadata['Price'].iloc[i]=price
        book_metadata['Rating'].iloc[i]=rating
        book_metadata.to_csv(my_csvfile,encoding='utf-8') #save data after each query
#     book_metadata['Description']=book_descriptions
#     book_metadata['Price']=book_price
#     book_metadata['Rating']=book_rating
    
    return book_metadata

<h2> Getting book Description </h2>

In [ ]:
def get_book_description(url,user_agent_list):
    book_description=''
    book_rating=''
    book_price=''
    print(url)
    #User Agents to prevent bot detection
    user_agent = random.choice(user_agent_list)
    headers = {'User-Agent': user_agent}
    
    #Throttling to spoof the server into believe these are generated by users
    time.sleep(7)
    
    book_description_response=requests.get(url,headers=headers)
    if book_description_response.status_code!=200:
        print("Error in scraping description")
        return None
    else:
        book_page=bs(book_description_response.content,"lxml")
        a1=book_page.find('div',{'id':'a-page'})
        a2=a1.find('div',class_='book en_US').find('div',class_='a-container').find('div',{'id':'centerCol'})
        try:
            book_description=a2.find('div',{'id':'bookDescription_feature_div'}).find('noscript').find('div').get_text()
        except:
            book_description=np.NAN
        try:
            book_rating=a2.find('div',{'id':'averageCustomerReviews_feature_div'}).find('span',class_='a-declarative').find('span',{'id':'acrPopover'}).find('span',class_='a-declarative').find('a').find('i').find('span').get_text().split()[0]    
        except:
            book_rating=np.NAN
        try:
            book_price=a2.find('div',{'id':'MediaMatrix'}).find('div').find('div',{'id':'tmmSwatches'}).find('ul').find('li',class_='swatchElement selected').find('span').find('span').find('span').find('span',class_='a-color-base').find('span').get_text().strip()
        except:
            book_price=np.NAN
        return book_description,book_price,book_rating

<h3> Combine in all AZscrape batches 1-7</h3>

In [ ]:
import os
import pandas as pd

path_dir = r'C:\Users\sanjayshah\MSBA Coursework\Tools for Analytics\datafile'
file_names = ['batch1.csv', 'batch2.csv', 'batch3.csv', 'batch4.csv', 'batch5.csv', 'batch6.csv', 'batch7.csv']
file_name_out = 'df_processed.csv'
path_output = os.path.join(path_dir, file_name_out)
fout = open(path_output, 'w', encoding='utf-8')

for batch in file_names:
    file_input = os.path.join(path_dir,batch)
    fin = open(file_input,'r',encoding='utf-8')
    for line in fin:
        fout.write(line)
fin.close()
fout.close()

<h3> Read in df_processed</h3>

In [ ]:
import pandas as pd
patha =  r'/Users/sanjayshah/Downloads/Tools_Analytics-master/df_processed.csv'
with open(patha):
    df=pd.read_csv(patha)
df = df.dropna(how='any',axis=0)

<h3> convert df['Rating'] values to floats and strings to 0's</h3>

In [ ]:
def ratings_to_floats(df):
    import re
    for i in df['Rating']:
        i = i.strip()
        m = bool(re.search('[a-zA-Z]',i))
        if i =='' or m==True:
            i='0.0'
        i = float(i)
    return df

<h2>convert df['Prices'] values to float</h2>

In [ ]:
def prices_to_floats(df):
    import re
    for i in df['Price']:
        i=i[1:]
        i=float(i)
        
    return df

<h2>Adding Word Cloud Segment</h2>

<h3><font color='red'>Please note</font> : This code has been tested on a subset</h3>

<h3>Remove the following words -</h3>
<li>Short Words</li>
<li>Redundant Words that appear in all descriptions</li>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS

#MIN_LENGTH and DELETE_WORDS will need to be updated once we identify the concerned words.

MIN_LENGTH=5
DELETE_WORDS=[]

def remove_words(text_string,DELETE_WORDS=DELETE_WORDS):
    for word in DELETE_WORDS:
        text_string = text_string.replace(word,' ')
    return text_string


def remove_short_words(text_string,min_length = MIN_LENGTH):
    word_list = text_string.split()
    for word in word_list:
        if len(word) < min_length:
            text_string = text_string.replace(' '+word+' ',' ',1)
    return text_string



## Categorize data
We're trying to group data into the following buckets based on the rating the book has been assigned
Group 1 : Ratings 5 - 4.5
Group 2 : Ratings 4.5 - 4
Group 3 : Ratings 4 - 3.5
Group 4 : Ratings 3.5 - 3
Group 5 : Ratings 3 - 2.5
Group 0 : Irrelevant to the analysis

In [ ]:
def categorize_data(df):
    
    #This list stores all the categories
    category_list=list()

    
    #Assign categories to each book
    
    for i in range(len(df)):
        if df['Rating'].iloc[i]>='4.5':
            category_list.append(1)
        elif df['Rating'].iloc[i]<'4.5' and df['Rating'].iloc[i]>='4':
            category_list.append(2)
        elif df['Rating'].iloc[i]<'4' and df['Rating'].iloc[i]>='3.5':
            category_list.append(3)
        elif df['Rating'].iloc[i]<'3.5' and df['Rating'].iloc[i]>='3':
            category_list.append(4)
        elif df['Rating'].iloc[i]<'3' and df['Rating'].iloc[i]>='2.5':
            category_list.append(5)
        else:
            category_list.append(0)

    #Add categories to the DF
    df['Category']=category_list
    
    return df

<h2>Code to aggregate text per rating</h2>

In [ ]:
def aggregate_text(temp):
    x=''
    for i in range(len(temp)):
        x=x+' '+temp['Description'].iloc[i]
    return x.lower()

<h2>Control Loop</h2>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS


import numpy as np
from PIL import Image

book_mask = np.array(Image.open("book.png"))

df = pd.read_csv('file1.csv')
df=df.dropna(how='any',axis=0)

COL_NUM = 2
ROW_NUM = 3
fig, axes = plt.subplots(ROW_NUM, COL_NUM, figsize=(15,15))



df=categorize_data(df)

#These two lists are for LSI Modeling
rating_list=list()
rating_description=list()

#Please refer to LSI Modeling.ipynb for more details. Code needs to be integrated.

for i in sorted(df['Category'].unique()):
    temp=df[df['Category']==i]
    text=aggregate_text(temp)
    text=remove_words(text)
    text=remove_short_words(text)
    rating_list.append(i)
    rating_description.append(text)
    ax = axes[i//2, i%2] 
    ax.set_title(i)
    wordcloud = WordCloud(stopwords=STOPWORDS,background_color='white',width=1200,height=1000,max_words=30,contour_width=20,mask=book_mask,contour_color='black').generate(text)
    ax.imshow(wordcloud)
    ax.axis('off')
plt.show()

<h2>Chart Distribution of Rating</h2>



In [ ]:
def chart_rating_distribution(df):
    %matplotlib inline
    import matplotlib.pyplot as plt
    import seaborn as sns
    plt.style.use('seaborn')
    from scipy import stats

    sns.distplot(df['Rating'],fit=stats.gamma)

<h2>Top 15 Top and Bottom Publishers by Rating</h2>

In [ ]:
import pandas as pd
path = r'C:\Users\endwy\Documents\Columbia MSBA\Fall 2018\E4501 - Tools for Analytics\Project\repo v3\df_processed.csv'
with open(path):
    import numpy as np
    df = pd.read_csv(path,keep_default_na=False)   
    
def table_top_and_bottom_publishers_by_rating(df):
    bottom = df.groupby('PUBLISHER', as_index=False)['Rating'].mean().nsmallest(15,'Rating')
    top = df.groupby('PUBLISHER', as_index=False)['Rating'].mean().nlargest(15,'Rating')
    return top,bottom

<h2>Number of Books per Publisher (Top 15)</h2>

In [ ]:
def num_books_per_publisher(df):
    from collections import Counter
    counts = dict(Counter(df['PUBLISHER']).most_common(15))
    pubs = list(counts.keys())
    nums = list(counts.values())

    import matplotlib.pyplot as plt 
    %matplotlib inline
    import numpy as np
    y_pos = np.arange(len(pubs))
    plt.barh(y_pos,nums)
    plt.title('Number of Books per Publisher (Top 15)')
    plt.yticks(y_pos,pubs)

    return plt.show()

<h1>Proportion of books in each Category(as per rating)</h1>

In [ ]:
df1=df.copy()
df1=categorize_data(df1)
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

def draw_Pie(dataframe, name):
    
    
    tmp = dataframe.groupby(name).size()

    # draw histgram
    data = [go.Pie(values = tmp.values, 
               labels = tmp.index)]

    # add title
    layout = go.Layout(title = 'Pie of ' + name,
                      margin = dict(l = 80, r = 100, t = 40, b = 40))

    # draw fig
    fig = go.Figure(data = data, layout = layout)

    iplot(fig, filename = name)
draw_Pie(df1, "Category")